In [2]:
# ------------------------------------------------------------
# Object detection (image classification) using Transfer Learning
# ------------------------------------------------------------

# (a) Load in a pre-trained CNN model trained on a large dataset
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

train_ds = tf.keras.utils.image_dataset_from_directory(
    'image_dataset',
    validation_split=0.2, subset='training', seed=123,
    image_size=(224, 224), batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'image_dataset',
    validation_split=0.2, subset='validation', seed=123,
    image_size=(224, 224), batch_size=32
)

train_ds = train_ds.map(lambda x, y: (x/255.0, y))
val_ds  = val_ds.map(lambda x, y: (x/255.0, y))


# (a) Load pre-trained CNN (VGG16 on ImageNet) without top classifier
base = VGG16(weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
             include_top=False, input_shape=(224, 224, 3))


# (b) Freeze lower convolutional layers
base.trainable = False


# (c) Add custom classifier with trainable parameters
model = Sequential([
    base,
    GlobalAveragePooling2D(),
    Dense(5, activation='softmax')  # Assume 5 classes
])


# (d) Train classifier layers on task data
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=5)


# (e) Fine-tune hyperparameters + unfreeze more layers as needed
base.trainable = True   # unfreeze all layers
model.compile(optimizer='adam',   # lower LR
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=2)


Found 240 files belonging to 5 classes.
Using 192 files for training.
Found 240 files belonging to 5 classes.
Using 48 files for validation.
Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.2604 - loss: 1.6384 - val_accuracy: 0.1250 - val_loss: 1.6815
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.4375 - loss: 1.4707 - val_accuracy: 0.2500 - val_loss: 1.5931
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.4583 - loss: 1.3795 - val_accuracy: 0.3125 - val_loss: 1.5357
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.6042 - loss: 1.3120 - val_accuracy: 0.4375 - val_loss: 1.4821
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.6719 - loss: 1.2462 - val_accuracy: 0.4583 - val_loss: 1.4285
Epoch 1/2
6/6 ━━━━━━━━━━━━━━━━━━━━ 71s 11s/step - accuracy: 0.3438 - loss: 295.4182 - val_accuracy: 0.2083 - val_loss: 2.5735
Epoch 2/2
6/6 ━━━━━━━━━━━━━━━━━━━━ 65s 11s/step - accuracy: 0.2135 - loss: 1.9321 - val_accuracy: 0.1250 - val_loss: 1

In [8]:
!python -m pip install --user tensorflow
